In [32]:
import comm as cmm
import codecs
import numpy as np
from string import Template

def fmtOrderBy(v):
	return "" if(v=="") else f"""\t\t\tCASE WHEN @SortColumn = '{v}' AND @SortOrder = 'ASC' THEN f.{v} END ASC,\n\t\t\tCASE WHEN @SortColumn = '{v}' AND @SortOrder = 'DESC' THEN f.{v} END DESC,"""

def fmtUpdateSet(v):
    return "" if(v=="") else f",{v} = @{v}"

def fmtInsertCols(v):
    return "" if(v=="") else f",{v}"

def fmtInsertVals(v):
    return "" if(v=="") else f",@{v}"

def CharN(v, n):
    tmp = '{:<'+ str(n) +"}"
    return tmp.format(v)


def fmtStrQuote(v):
    # print(v, type(v))
    # return str(v) if(type(v)==int or type(v) == float) else f"'{str(v)}'"
    return str(v) if(str(v).isnumeric()) else f"'{str(v)}'"

def fmtCmmt_ParasInput(r):
    col, AtCol, typLen, ColDscr = r["Column"], r["AtCol"], r["typLen"], r["ColDscr"]
    # typLen = "VARCHAR(10)" if (col == "IsEnable") else typLen
    AtCol = "{:<16}".format(AtCol)
    typLen   = "{:<12}".format(typLen)
    return f"{AtCol} {typLen} - {ColDscr}"

def fmtCmmt_ParasInputQuery(r):
    col, AtCol, typLen, ColDscr = r["Column"], r["AtCol"], r["typLen"], r["ColDscr"]
    
    typLen = "VARCHAR(10)" if (col == "IsEnable") else typLen
    AtCol = "{:<16}".format(AtCol)
    typLen   = "{:<12}".format(typLen)
    return f"{AtCol} {typLen} - {ColDscr}"

def fmtCmmt_returnSelect(r):
    col, typLen, colDscr = r["Column"], r["typLen"], r["ColDscr"]
    col = "{:<16}".format(col)
    typLen   = "{:<12}".format(typLen)
    return f"** {col} {typLen} - {colDscr}"

def fmtTableCompos(r):
    col, typLenTbOnly, autoGen, isNull = r["Column"], r["typLenTbOnly"], r["autoGen"], r["isNull"]
    isNull = "NULL" if(isNull == "V") else "NOT NULL"
    autoGen = "IDENTITY(1,1)" if (autoGen =="V") else ""
    return f"\t[{col}] {typLenTbOnly} {autoGen}{isNull},"

def fmtTableExt(r):
    schema, tableName, col, colDscr, colDscrMemo =  r["Schema"], r["TableName"], r["Column"], r["ColDscr"], r["colDscrMemo"]
    colDscrMemo = "" if (colDscrMemo =="") else f" : {colDscrMemo}"
    colDscr = colDscr + colDscrMemo
    return f"EXEC sys.sp_addextendedproperty @name=N'MS_Description', @value=N'{colDscr}' ,@level0type=N'SCHEMA',@level0name=N'{schema}', @level1type=N'TABLE',@level1name=N'tb{tableName}', @level2type=N'COLUMN',@level2name=N'{col}'\nGO"

def fmtCmmDeclares(r):
    atCol, typLen = r["AtCol"], r["typLen"]
    return f",{atCol} {typLen}"

def fmtCmmDeclaresQuery(r):
    col, atCol, typLen = r["Column"], r["AtCol"], r["typLen"]
    typLen = "VARCHAR(10)" if(col == "IsEnable") else typLen
    return f",{atCol} {typLen}"

def fmtSetValue(r):
    AtCol, v, typ = r["AtCol"], r["spDelete"], r["Type"]
    v = int(v) if(typ in ["int","decimal","float"] and v != "") else f"'{v}'"
    return f"SET {AtCol} = {v}"

def fmtExistSetValue(r):
    AtCol, v, typ = r["AtCol"], r["spExist"], r["Type"]
    v = int(v) if(typ in ["int","decimal","float"] and v != "") else f"'{v}'"
    return f"SET {AtCol} = {v}"

def fmtInsertSetValue(r):
    AtCol, v, typ = r["AtCol"], r["spInsert"], r["Type"]
    v = int(v) if(typ in ["int","decimal","float"] and v != "") else f"'{v}'"
    return f"SET {AtCol} = {v}"

def fmtUpdateSetValue(r):
    AtCol, v, typ = r["AtCol"], r["spUpdate"], r["Type"]
    v = int(v) if(typ in ["int","decimal","float"] and v != "") else f"'{v}'"
    return f"SET {AtCol} = {v}"

def fmtQuerySetValue(r):
    AtCol, v, typ = r["AtCol"], r["spQueryWhere"], r["Type"]
    v = int(v) if(typ in ["int","decimal","float"] and v != "") else f"'{v}'"
    return f"SET {AtCol} = {v}"

def fmtQueryWhere(r):
    tmp, col = "", r["Column"]
    t1 =f"AND f.$col LIKE CASE WHEN @$col = '' THEN f.$col ELSE '%' + @$col + '%' END"
    t2 =f"AND f.IsEnable = CASE WHEN @IsEnable = 'ALL' THEN f.IsEnable ELSE CASE WHEN @IsEnable = '1' THEN 1 ELSE 0 END END"
    tmp = t2 if(col == "IsEnable") else t1
    tmp = Template(tmp)
    return tmp.substitute(col = col) 

In [34]:
import pandas as pd
import numpy as np
import pyodbc
import comm as cmm
import time
import codecs
from string import Template

#指定Excel表路徑名稱。
_ExcelFile = "AGD_DBSchema_20220410_V10.xlsx"
pd.options.mode.chained_assignment = None  # default='warn'

df = pd.read_excel(_ExcelFile, sheet_name="admin_columns",converters={'maxLen':str }).replace(np.nan, '')
df_sp = pd.read_excel(_ExcelFile, sheet_name="SP_config",converters={'maxLen':str }).replace(np.nan, '')

# for basic transformation. 基本型態
df["fCol"]          = ",f." + df["Column"]                     #格式：,f.ExtCode
df["AtCol"]         = "@" + df["Column"]                       #格式：@ExtCode
df["Typ"]           = df["Type"].str.upper()                   #格式：varchar => VARCHAR  轉換為大寫。
df["typ"]           = df["Type"].str.lower()                   #格式：varchar => varchar  轉換為小寫。
df["typLen"]        = df["Typ"] + df["maxLen"].apply(cmm.fmtParentheses)                #格式：VARCHAR(20)
df["typLenTbOnly"]  = "[" + df["typ"] + "]" + df["maxLen"].apply(cmm.fmtParentheses)    #格式：[varchar](20)
df["colEqualAt"]    = df["Column"] + " = " + df["AtCol"]       # SeqNo = @SeqNo  
df["colAtEqualAt"]  = df["AtCol"] + " = " + df["AtCol"]        # @SeqNo = @SeqNo
df["AtParasSet"]    = "@" + df["Column"] + " " + df["typLen"]
df["cmmt_Exec"]     = "," + df["AtCol"]  + " = " + df["AtCol"] 

# for create Tables...
df["colPKs"]                = "\t["+df["Column"]+"] ASC,"
df["colComp"]               = df.apply(fmtTableCompos, axis = 1)
df["colExt"]                = df.apply(fmtTableExt, axis = 1)
# for create store procedure.
df["parasSet"]              = df["Column"] + " " + df["typLen"]

df["cmmt_InputParas"]       = df.apply(fmtCmmt_ParasInput, axis = 1)
df["cmmt_InputParasQuery"]  = df.apply(fmtCmmt_ParasInputQuery, axis = 1)
df["cmmt_Declares"]         = df.apply(fmtCmmDeclares, axis =1)
df["cmmt_DeclaresQuery"]    = df.apply(fmtCmmDeclaresQuery, axis =1)
df["cmmt_select"]           = df.apply(fmtCmmt_returnSelect, axis = 1)
df["orderBy"]               = df["Column"].apply(fmtOrderBy)
df["SetValue"]              = df.apply(fmtSetValue, axis = 1)
df["existSetValue"]         = df.apply(fmtExistSetValue, axis = 1)
df["insertSetValue"]        = df.apply(fmtInsertSetValue, axis = 1)
df["updateSetValue"]        = df.apply(fmtUpdateSetValue, axis = 1)
df["querySetValue"]         = df.apply(fmtQuerySetValue, axis = 1)

df["queryWhere"]            = df.apply(fmtQueryWhere,  axis =1)  # f.{v} LIKE CASE WHEN @{v} = '' THEN f.{v} ELSE '%' + @{v} + '%' END



def GenCreateTable(df):
    tbNames = df["TableName"].unique()
    for tb in tbNames:
        colsStr, ColExtFormats="",""
        dfs = df[df["TableName"]==tb]
      
        tb_colsPK = "\n".join(dfs[dfs["PK"]=="V"]["colPKs"].tolist())[:-1]
        colsStr = "\n".join(dfs["colComp"].tolist())
        ColExtFormats = "\n".join(dfs["colExt"])

        tmpFile = codecs.open("template/SQL/createTable.sql","r").read().format(tb=tb, tb_colsPK = tb_colsPK, colsStr=colsStr, ColExtFormats=ColExtFormats)
        fileName = f"tables/tb{tb}.sql"
        os.makedirs(os.path.dirname(fileName), exist_ok=True)
        f = codecs.open(fileName, "w","utf-8")
        f.write(tmpFile)
        f.close()
        
GenCreateTable(df)  ## 自動產生Tables

_sq = "SeqNo"

def GenSP(tb, _df, tmpFileTag, spType):
    tbDscr = df_sp[df_sp["TableName"] == tb]["tbDscr"].values[0]

    ### 從Excel內選擇資料來源，請對照Excel檔案，只要不為空白就是要參照到Template中。 ###
    df = _df if(spType == "") else _df[_df[spType] != ""] 

    pt_getSelectAll = "\n".join(_df["cmmt_select"].tolist())   
    pt_queryWherePK = " AND ".join(df[df["Column"] == _sq]["colEqualAt"].tolist())
    pt_cmmtExecPK   = "\n\t\t,".join(df[df["Column"] == _sq]["colAtEqualAt"].tolist())
    pt_parasSetPK   = "\n,".join(df[df["Column"] == _sq]["parasSet"].tolist())
    pt_AtParasSetPK = "\n,".join(df[df["Column"] == _sq]["AtParasSet"].tolist())
    # print(pt_parasSetPK)
    pt_inputPK      = "\n\t".join(df[df["Column"] == _sq]["cmmt_InputParas"].tolist())
    pt_DeclarePK    = "\n\t\t".join(df[df["Column"] == _sq]["cmmt_Declares"])[1:]
    pt_deleteSetVal =  "\n\t".join(df[df["Column"] == _sq]["SetValue"].tolist())
    pt_fColOr = " OR\n\t\t\t\t".join(df[df["Column"] != _sq]["colEqualAt"].tolist())  #不為PK的通通算進去

    pt_select       = "\n".join(df["cmmt_select"].tolist())
    pt_input        = "\n\t".join(df["cmmt_InputParas"].tolist())
    pt_inputQuery   = "\n\t".join(df["cmmt_InputParasQuery"].tolist())
    pt_Declare      = "\n\t".join(df["cmmt_Declares"])[1:]
    pt_DeclareQuery = "\n\t".join(df["cmmt_DeclaresQuery"])[1:]
    
    pt_fCol         = "\n\t\t\t".join(df["fCol"])[1:]
    pt_fColAll      = "\n\t\t\t".join(_df["fCol"])[1:]
    
    
    pt_orderBy ="\n".join(_df[_df["spOrderBy"] == "V"]["orderBy"].tolist())[:-1]

    pt_Exec = "\n\t\t".join(df["cmmt_Exec"])[1:]
    pt_queryWhere = "\n\t\t\t".join(df["queryWhere"].tolist())[4:]
    pt_UpdateSet = "\n\t\t\t,".join(df[(df["Column"]!="SeqNo") ]["colEqualAt"].tolist())
    pt_insertCols = "\n\t\t\t,".join(df["Column"].tolist())
    pt_insertVals = "\n\t\t\t".join(df["Column"].apply(fmtInsertVals).tolist())[1:]
    
    pt_existSetValue ="\n\t".join(df[df["spExist"] != ""]["existSetValue"].tolist()) 
    pt_insertSetVal = "\n\t".join(df[df["spInsert"] != ""]["insertSetValue"].tolist())
    pt_updateSetVal = "\n\t".join(df[df["spUpdate"] != ""]["updateSetValue"].tolist())
    pt_querySetVal  = "\n\t".join(df[df["spQueryWhere"] != ""]["querySetValue"].tolist())


    tmpFile = codecs.open(f"template/SQL/agdSP.{tmpFileTag}.sql",mode="r",encoding="utf-8",errors='strict', buffering=-1).read()
    ## 開始套表
    tmpFile = Template(tmpFile).substitute(pt_tableName = tb, pt_tbDscr = tbDscr, pt_delete = "Delete", pt_exists ="Exists", pt_get = "Get", pt_query = "Query", pt_update = "Update", pt_insert = "Insert", pt_DateTime = time.strftime("%Y/%m/%d %H:%M:%S", time.localtime()), pt_getSelectAll = pt_getSelectAll, pt_inputPK = pt_inputPK, pt_queryWherePK = pt_queryWherePK ,pt_cmmtExecPK = pt_cmmtExecPK, pt_DeclarePK= pt_DeclarePK, pt_parasSetPK = pt_parasSetPK, pt_deleteSetVal = pt_deleteSetVal, pt_select = pt_select, pt_input = pt_input,pt_Declare = pt_Declare, pt_DeclareQuery = pt_DeclareQuery, pt_inputQuery = pt_inputQuery, pt_Exec = pt_Exec, pt_fCol = pt_fCol, pt_fColOr = pt_fColOr, pt_fColAll = pt_fColAll, pt_orderBy = pt_orderBy, pt_queryWhere = pt_queryWhere, pt_UpdateSet = pt_UpdateSet, pt_insertCols = pt_insertCols, pt_insertVals = pt_insertVals, pt_insertSetVal = pt_insertSetVal, pt_updateSetVal = pt_updateSetVal, pt_querySetVal = pt_querySetVal, pt_existSetValue = pt_existSetValue, pt_AtParasSetPK = pt_AtParasSetPK)

    # tb2 = "Aux2" if (tb == "Aux") else tb 
    fileName = f"agdSP/usp{tb}{tmpFileTag}.sql"
    # print(fileName)
    os.makedirs(os.path.dirname(fileName), exist_ok=True)
    f = codecs.open(fileName, mode="w", encoding="utf-8", errors="strict")
    f.write(tmpFile)
    f.close()

tbs = df_sp["TableName"].unique()

for tb in tbs:
    
    # if(tb in ["PcPhone"]):
    _df = df[df["TableName"] == tb]

    GenSP(tb, _df, "Delete", "")
    GenSP(tb, _df, "Exists", "spExist")
    GenSP(tb, _df, "Get",    "spDelete")
    GenSP(tb, _df, "Query",  "spQueryWhere")
    GenSP(tb, _df, "Update", "spUpdate")
    GenSP(tb, _df, "Insert", "spInsert")
    